# From Jupyter Notebook to Web Server: Automating Container Deployments with GitHub Actions

<strong>The purpose of this notebook is to provide an example of how one could use GitHub Actions to build a container image that could be hosted on the web via Helm charts. The hope is that what is learned can be applied to work that is interesting to you and open up avenues to create containers with interactive web content.</strong>

## Prerequisites

This walkthrough requires a [GitHub account](https://github.com/signup?ref_cta=Sign+up&ref_loc=header+logged+out&ref_page=%2F&source=header-home), a [Docker Hub account](https://hub.docker.com/signup), and for the example repository to be [forked](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo). 

## An Introduction to Containers

Containers package software and dependencies into a single image that can be run on any platform with a container engine installed. The image is portable and immutable, the software inside won't change. It will reliably run in the same state years from now making it powerful for easy reproducibility. 

The walkthrough from the June 20th, 2024 workshop covers what containers are in greater detail. For more information, see [https://github.com/Gin-G/nbviz-to-container-gin-g/blob/main/June-20-2024/walkthrough.ipynb](From Jupyter Notebook to Web Server: Containerizing Interactive Visualizations)

The container built during the June 20th workshop had to download data that caused timeouts for some users with slower internet connections. There is a Containerfile in this directory that contains a different web application that applies the same principles. It creates a smaller image that loads the website faster and should not timeout. Each command/layer is explained in comments inside the containerfile. 

## Getting Started

We are going to work out of the browser for majority of this workshop. GitHub provides an Integrated Development Environment (IDE) that can load any repository and allows for editing directly in a web browser. 

Open this repository, [https://github.com/NicholasCote/nbviz-to-container](nbviz-to-container), in your browser of choice. The body of the page contains a button in the upper right hand corner to Fork the repository. Click the Fork button to bring up the Create a new fork page. From here we can select an Owner, you, and name the repository. To avoid confusion in the future it's best to append the repository name with something like you GitHub user account name. All the other defaults are fine and we can select the Create fork button at the bottom of the page. This creates a copy of the repo contents under your own GitHub account. 

**Note:** If you attended the previous workshop we forked this repository and an existing fork may already exist. If you find yourself in the situation we can try to sync your existing fork or delete the old fork and create a new one. 

Once the fork is complete we will be on a GitHub page with the repository contents, but under our own accounts. The next step is going to be to open the repository in [github.dev](https://github.dev/). The easiest way to do this is to change the .com to .dev in the full URL displayed in our browser and hit Enter. We should now be in the GitHub IDE and can work inside the repository. 

## On-premise cloud deployment and automation

Once there is a working container image that runs a web server locally it can be used in a number of different places, including as the content for a website at a live URL. Automation with GitHub Actions can be utilized to build a new image when changes are made and update configuration files, known as a [Helm chart](https://helm.sh/), to use the newest image. The Helm chart directory in the repository can be connected to a Continuous Delivery tool called [Argo CD](https://argoproj.github.io/cd/) which enables automatic synchronization The NSF NCAR CISL On-premise Cloud Pilot is setup to host these kinds of workloads and only requires a set of configuration files known as a [Helm chart](https://helm.sh/).

### GitHub Actions

A [GitHub Actions](https://docs.github.com/en/actions) workflow is defined as a YAML file and can be created to build a container when changes are made to a repository. This job can be run on GitHub-provided machines, called runners, or self-hosted runners that can be run on a users machine or a central compute system like the CISL On-premise Cloud Pilot. YAML files define each workflow and are stored at the repository's base in a directory named `.github/workflows/`. On the left side bar in github.dev add a new folder named .github to the repository base by either right-clicking and selecting New Folder or clicking the new folder icon at the top of the sidebar. Inside the new .github folder add a new folder named workflows. This new location will be where we define how to run specific GitHub Actions. 

#### GitHub provided runner example

##### Build a container image

Sometimes it is advantageous to build an image, but not push it anywhere. For instance, you may want to test that the changes made don't affect the image build and it's still able to build completely without errors. We are going to start with this example before moving on to a workflow that pushes an image. Inside the `.github/workflows/` directory create a new file where we can define the GitHub Action. Use a descriptive name like `webapp-build-test.yaml` when creating the new file. The contents to place inside the file can be found below with explanations of each line, or group of lines, included as comments:

```yaml
# Name the Action
name: Automation to build a container image
# Define when the Action is run. This example is run when there is a push to the flask-app/ directory on the app branch.
on:
  push:
    paths:
      - August-08-2024/**
    branches:
      - main
# Define the jobs to run. A job can have multiple steps and an Action can contain multiple jobs. 
jobs:
  build-image:
    # Use the latest ubuntu image to run the jobs
    runs-on: ubuntu-latest
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v4
      # Get the date and send to GITHUB_OUTPUT to apply as image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Build the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the current directory as build context 
          context: August-08-2024/.
          # Specify where the Dockerfile is located in relation to the repo base path
          file: August-08-2024/Containerfile
          # Enable the push to docker hub
          push: false
          # Provide the tags to apply to the image, this example uses the latest image tag 
          tags: |
            ncote/workshop-webapp:${{ steps.date.outputs.date }}
```

Push the changes, the new workflow action file, by using the branch on the left side of your browser window. It should have a circle with a 1 inside it to show we have 1 change that can be committed and pushed. After clicking on the branch icon there will be a text box to enter a commit message. Enter a descriptive message like added build test workflow file and click the green Commit & Push button. Our Action is now live in our repository and we can use it to build images. 

##### Test the new workflow

The GitHub action defined only triggers when changes are made to the August-08-2024 directory on the main branch. Let's open the Containerfile and add a new comment so it changes the Containerfile, but not actually how it's built and run. Once the comment has been added you will see that the GitHub extension in the left bar shows we have another change that can be pushed. Let's go ahead and add another commit message and push the new changes to kick off the GitHub Action. Once the push has been made we can watch the status of our GitHub action by going back to teh repository URL and selecting Actions in the middle of the top navigation bar on the site.  

##### Build and push a container image

Once we know the image is building correctly we can go ahead and push that image to a container registry so it can be shared with others or reused. The image created in the example is pushed to [Docker Hub](https://hub.docker.com/) and uses credentials stored as repository [Actions secrets](https://docs.github.com/en/actions/security-guides/using-secrets-in-github-actions) to log in. If we go back to the code repository in our browser we can open up the repository settings by using the far right option on the top navigation bar. Once in the repo security settings we can look at the left navigation menu that is opened and under Security there is a dropdown for Secrets and variables that contains a link to Actions. Once we have the Actions repository secrets open we can add our Docker Hub information based on the variables used in the workflow file, DOCKERHUB_USERNAME & DOCKERHUB_TOKEN. Copy and paste the contents below into our existing GitHub Actions workflow file and overwrite everything that was in there before. 

```yaml
# Name the Action
name: Automation to build a container image
# Define when the Action is run. This example is run when there is a push to the flask-app/ directory on the app branch.
on:
  push:
    paths:
      - August-08-2024/**
    branches:
      - main
# Define the jobs to run. A job can have multiple steps and an Action can contain multiple jobs. 
jobs:
  build-image:
    # Use the latest ubuntu image to run the jobs
    runs-on: ubuntu-latest
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v4
      # Step 2 is to login to docker hub so the image can be pushed
      - name: Login to Docker Hub
        uses: docker/login-action@v2
        # GitHub repository secrets are used as variables to provide login information to Docker Hub
        # DOCKERHUB_USERNAME and DOCKERHUB_TOKEN need to be added to the Actions secrets
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN  }}
      # Get the date and send to GITHUB_OUTPUT to apply as image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Build and push the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the current directory as build context 
          context: .
          # Specify where the Dockerfile is located in relation to the repo base path
          file: Containerfile
          # Enable the push to docker hub
          push: true
          # Provide the tags to apply to the image, this example uses the latest image tag 
          tags: |
            ncote/nbviz2cntnr:${{ steps.date.outputs.date }}
```

#### Self-hosted runner example

```{note}
The self-hosted runner example depends on resources that currently are only available on the UCAR internal network. A VPN connection or access onsite at a UCAR facility is required to successfully run the commands and Action example. 
```

I have built a custom container image to do rootless podman builds and the configuration information can be found at this [link to my github-runner repository](https://github.com/NicholasCote/github-runner). The up to date container image is stored on the NSF NCAR CISL On-premise Cloud private container registry running on Harbor. In order to run the self-hosted runner on your local machine a GitHub API token is required when the container is launched. If you do not have an existing token use this [link to create an API token](https://github.com/settings/tokens). The example below uses a local environment variable, ${GITHUB_RUNNER_TOKEN}, instead of supplying the token as plain text in the command:

`podman run -e REPO={user/org}/{repo_name} -e TOKEN=${GITHUB_RUNNER_REGISTRATION} hub.k8s.ucar.edu/ncote/github-runner:2024-06-04.00.56`

Replace {user/org}/{repo_name} with the repository you want to connect the runner to, i.e. NicholasCote/nbviz-to-container. When run this will pull down the runner container image and start it automatically. There are a few prompts that will come up, the default value can be used for each one. Once those settings are entered there will be a confirmation that the runner is connected to GitHub and is Listening to Jobs. A GitHub action workflow that utilizes self-hosted runners can now be added and executed. Explanations of each line, or group of lines, are provided as comments in the example seen below:

```yaml
# This workflow builds container images and pushes them to the NSF NCAR Harbor Container Repository
# Set the workflow name
name: GitHub Runner Build, Push, & Update

# Define the trigger that starts the action
# For this workflow the trigger is on a push that changes anything in the main branch
on:
  - push

# Define the actions that are going to take place as part of this workflow    
jobs:
  # Name the job(s)
  build-push-gh-runner-cicd:
    # Define where the job should run in this case it will be run on the self-hosted runner image
    runs-on: self-hosted
    # Set the steps to take in order
    steps:
      # Step 1 is to checkout the github repo used to build the container image
      - name: Check out the repo
        uses: actions/checkout@v4
      # Get the date to apply to image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H.%M')" >> $GITHUB_OUTPUT
      # Use podman to build the image and tag it with the current date
      - name: Build and push image
        run: podman build -t hub.k8s.ucar.edu/ncote/nbviz2cntnr:${{ steps.date.outputs.date }} .
      # Login to Harbor with a robot account for the ncote project in Harbor
      - name: Login to Harbor
        run: podman login hub.k8s.ucar.edu -u robot-ncote+ncote -p ${{ secrets.HARBOR_ROBOT_PW  }}
      # Push the image to Harbor
      - name: Push image to Harbor
        run: podman push hub.k8s.ucar.edu/ncote/nbviz2cntnr:${{ steps.date.outputs.date }}
```

```{note}
In this example, Workflow permissions need to be changed in the GitHub repository Settings under Actions -> General. Update the Workflow permissions to be Read and write to allow the Action to make changes to the repository.
``` 

This workflow will run whenever there's changes and build a new container image. That image will be pushed to an internal container registry and the applications Helm chart will be updated to use the new image. If the Helm chart is connected to the CISL On-premise clouds [Argo CD instance](https://ncar.github.io/cisl-cloud/how-to/K8s/Hosting/deploy.html) it will automatically sync the changes and the site will be updated a few minutes after the changes have been made. 

### Helm chart

In order to run the container on the CISL On-premise Cloud Pilot, a Kubernetes (K8s) cluster, a collection of 3 different K8s objects need to be created. The objects can be bundled together in to a Helm chart, a collection of templates that use custom values to define an applications. The 3 different K8s objects that are required are a [deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), [service](https://kubernetes.io/docs/concepts/services-networking/service/), and [ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/). A K8s deployment is the actual application running in 1 or more Pods. The K8s service exposes the Pod to the K8s internal network. Lastly the K8s ingress maps the service to a FQDN and exposes it outside the cluster. There are shared services running on the K8s cluster that will assign DNS records and valid certificates to provide a resolvable URL that is secured via HTTPS. In a Helm chart the K8s object definitions can be setup to take variables defined in a values.yaml file allowing for customization. 

#### Simple Web App Template

There is a simple web application Helm chart template available at this [link to web-app-helm repository](https://github.com/NicholasCote/web-app-helm). The web-app-helm directory included in the repo can be copied in to the repository that contains the container image and application code. Inside the web-app-helm directory is a file named values.yaml that needs to be updated with all the custom values required for your application. The README file included in the template repository contains instructions on what needs to be updated and what values to provide. A general overview of what is required to customize the values.yaml file is as follows:

1. A Name for your application, this is not the URL that will be deployed but the name of the k8s objects created
2. A URL path. This also is not the full URL, just the suffix you'd like to use after `.edu`. This is typically just `/` but may be things like `/api` that correspond to endpoints on your application.
3. An FQDN. This is the full URL for your application. Currently in the CISL cloud environment we only can create names under the `.k8s.ucar.edu` domain and your FQDN should end with `.k8s.ucar.edu`. Please make sure this is unique, try to browse to it before applying, and descriptive for your application. 
4. Container image to use. This should be an image that is already built and has been pushed to a container registry that the application can pull from. By default it is set to look at docker.io so if you are using something different you need to specify that before your container registry and image name:tag
5. Container port to expose. Your containerized application will expose a port to the network in order to communicate. More often than not there is a default for the application you are using and you also have the ability to provide a specific port if you wanted. If you have run your container image locally it is usually in the URL you used to access it locally, ie. `http://127.0.0.1:8888` is running on port 8888 and would be the appropriate value to put in the Helm chart. 